In [ ]:
import algoseek_connector as ac
from sqlalchemy import func
import numpy as np
import pandas as pd
import time

In [ ]:
# a ResourceManager creates connections to different data sources
# set access to the metadata API with the environment variables
# ALGOSEEK_API_USERNAME and ALGOSEEK_API_PASSWORD
manager = ac.ResourceManager()

In [ ]:
# a data source manages data groups avaiable in a data source
# Set access to ArdaDB using the environment variables
# ALGOSEEK_ARDADB_HOST, ALGOSEEK_ARDADB_USERNAME and ALGOSEEK_ARDADB_PASSWORD
data_source = manager.create_data_source("ardadb")

In [ ]:
data_source.groups.USEquityMarketData

In [ ]:
# DataGroup instances are created using the fetch method.
group = data_source.groups.USEquityMarketData.fetch()

In [ ]:
# a dataset is created using the DataSetFetcher fetch method.
dataset = group.datasets.TradeOnly.fetch()

In [ ]:
dataset

## Querying the dataset

In [ ]:
# a column handler contains the dataset columns
# columns can also be accessed using dataset attributes.
c = dataset.get_column_handle()

## 1- Simple select query

In [ ]:
# SQL statement objects are created using the dataset's select method.
# SQL statements are created using method chaining using utilities
# from SQLAlchemy
stmt = dataset.select().limit(10)

In [ ]:
# Using the compile method, the SQL statement created can be displayed as a formatted code block
query = dataset.compile(stmt)
query

In [ ]:
%%time
df = dataset.fetch_dataframe(stmt)

In [ ]:
df

In [ ]:
df.dtypes

## 2 - Select with group by

In [ ]:
agg_year_label = "year"
year_having_filter = 2015
stmt = (
    dataset.select(
        func.avg(dataset.Quantity).label("mean_quantity"),
        func.toYear(dataset.TradeDate).label(agg_year_label),
    )
    .group_by(func.toYear(dataset.TradeDate))
    .having(func.toYear(dataset.TradeDate) == year_having_filter)
)

In [ ]:
dataset.compile(stmt)

In [ ]:
%%time
result = dataset.fetch_dataframe(stmt)

In [ ]:
result.head()

## Streaming queries

In [ ]:
stmt = dataset.select(c["Quantity"], c["Ticker"]).limit(50)

In [ ]:
query = dataset.compile(stmt)
query

In [ ]:
query.parameters

In [ ]:
res = list()
for chunk in dataset.fetch_iter_dataframe(stmt, size=10):
    print(f"Columns: {list(chunk.columns)}, shape: {chunk.shape}")
    res.append(chunk)
    time.sleep(0.5)

In [ ]:
res[0]

## Working with the TAQ dataset

In [ ]:
dataset = group.datasets.TradeAndQuote.fetch()
dataset

In [ ]:
stmt = dataset.select().where((dataset["TradeDate"] == "20150303") & (dataset["Ticker"] == "AAPL"))

In [ ]:
dataset.compile(stmt)

In [ ]:
%%time
df = dataset.fetch_dataframe(stmt)
size_mb = df.memory_usage().sum() / (1024 * 1024)
print(f"data size = {size_mb:.1f} MiB")

In [ ]:
df.head()